In [ ]:
# 하이퍼 파라미터 튜닝

In [ ]:
1 교차검증

1.1 교차검증
과적합 : 모델이 학습 데이터에만 과도하게 최적화된 현상.
그로인해 일반화된 데이터에서는 예측 성능이 과하게 떨어지는 현상
지난번 와인 맛 평가에서 훈련용 데이터의 Acc는 72.94,
테스트용 데이터는 Acc가 71.61%였는데, 누가 이 결과가 정말 괜찮은 것인지 묻는다면?
나에게 주어진 데이터에 적용한 모델의 성능을 정확히 표현하기 위해서도 유용하다

In [ ]:
1.2 holdout

In [ ]:
2 교차 검증 구현하기

In [21]:
import numpy as np
from sklearn.model_selection import KFold

X= np.array([[1,2],[3,4],[1,2],[3,4]])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=2)

print(kf.get_n_splits(X))
print(kf)
for train_idx, test_idx in kf.split(X):
    print('---idx')
    print(train_idx, test_idx)
    print('---train data')
    print(X[train_idx])
    print('---val data')
    print(X[test_idx])

2
KFold(n_splits=2, random_state=None, shuffle=False)
---idx
[2 3] [0 1]
---train data
[[1 2]
 [3 4]]
---val data
[[1 2]
 [3 4]]
---idx
[0 1] [2 3]
---train data
[[1 2]
 [3 4]]
---val data
[[1 2]
 [3 4]]


In [ ]:
2.3 다시 와인 맛 분류하던 데이터로

In [9]:
import pandas as pd

red_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
white_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

red_wine = pd.read_csv(red_url, sep =';')
white_wine = pd.read_csv(white_url, sep =';')

red_wine['color'] = 1.
white_wine['color'] = 0.

wine = pd.concat([red_wine, white_wine])

2.4 와인 맛 분류기를 위한 데이터 정리

In [31]:
wine['taste'] = [1. if grade >5 else 0. for grade in wine['quality']]

X = wine.drop(['taste', 'quality'], axis = 1)
y = wine['taste']

2.5 지난번 의사 결정 나무 모델로는?

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 13)

wine_tree = DecisionTreeClassifier(max_depth=2, random_state=13)
wine_tree.fit(X_train, y_train)

y_pred_tr = wine_tree.predict(X_train)
y_pred_test = wine_tree.predict(X_test)

print('Train Acc : ', accuracy_score(y_train, y_pred_tr))
print('Test Acc : ', accuracy_score(y_test, y_pred_test))


Train Acc :  0.7294593034442948
Test Acc :  0.7161538461538461


2.6 KFold

In [33]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits = 5)
wine_tree_cv = DecisionTreeClassifier(max_depth = 2, random_state = 13)


2.7 KFold는 index를 반환한다

In [34]:
for train_idx, test_idx in kfold.split(X):
    print(len(train_idx), len(test_idx))

5197 1300
5197 1300
5198 1299
5198 1299
5198 1299


2.8 각각의 fold에 대한 학습 후 acc

In [35]:
cv_accuracy = []
for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    wine_tree_cv.fit(X_train, y_train)
    pred = wine_tree_cv.predict(X_test)
    cv_accuracy.append(accuracy_score(y_test, pred))

cv_accuracy

[0.6007692307692307,
 0.6884615384615385,
 0.7090069284064665,
 0.7628945342571208,
 0.7867590454195535]

In [36]:
np.mean(cv_accuracy)

0.709578255462782

2.10 StratifiedKFold

In [37]:
from sklearn.model_selection import StratifiedKFold

skfold = StratifiedKFold(n_splits = 5)
wine_tree_cv = DecisionTreeClassifier(max_depth = 2, random_state = 13)


cv_accuracy = []

for train_idx, test_idx in skfold.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    wine_tree_cv.fit(X_train, y_train)
    pred = wine_tree_cv.predict(X_test)
    cv_accuracy.append(accuracy_score(y_test, pred))

cv_accuracy

[0.5523076923076923,
 0.6884615384615385,
 0.7143956889915319,
 0.7321016166281755,
 0.7567359507313318]

In [38]:
np.mean(cv_accuracy)

0.6888004974240539

2.12 cross validation을 보다 간편히